In [57]:
!pip install transformers
!pip install datasets

import os
import re
import pandas as pd
import numpy as np

from sklearn import preprocessing

from transformers import AutoTokenizer
from datasets import Dataset
from transformers import DataCollatorWithPadding
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
from transformers import BertTokenizerFast

from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

import pandas as pd
from tqdm import tqdm

from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.nn import functional as F

import torch
from torch.utils.checkpoint import checkpoint
import torch.nn as nn

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

TRAIN_NOQUANT = True
QUANT_TRAIN = True

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
# You can change the model name here, look up model names from huggingface docs
model_name = "bert-base-multilingual-cased"
batch_size_per_device = 32
max_length = 64
n_epochs = 3
warmup_ratio = .2
lr = 5e-5

In [59]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [60]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=7)
model.to(device)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attent

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [61]:
class TorchQuantize(nn.Module):
    """ 
    Quantize an input tensor to the fixed-point representation. 
        Args:
        input: Input tensor
        bits:  Number of bits in the fixed-point
    """
    def __init__(self, bits=0):
        super(TorchQuantize, self).__init__()
        if bits == 0:
            self.quantize = nn.Identity()
        elif bits == 1:
            self.quantize = TorchBinarize()
        else:
            self.quantize = TorchRoundToBits(bits)

    def forward(self, input):
        return self.quantize(input)


In [62]:
class TorchBinarize(nn.Module):
    """ Binarizes a value in the range [-1,+1] to {-1,+1} """
    def __init__(self):
        super(TorchBinarize, self).__init__()

    def forward(self, input):
        """  clip to [-1,1] """
        input = Clamp.apply(input, -1.0, 1.0)
        """ rescale to [0,1] """
        input = (input+1.0) / 2.0
        """ round to {0,1} """
        input = Round.apply(input)
        """ rescale back to {-1,1} """
        input = input*2.0 - 1.0
        return input

In [63]:
class TorchRoundToBits(nn.Module):
    """ Quantize a tensor to a bitwidth larger than 1 """
    def __init__(self, bits=2):
        super(TorchRoundToBits, self).__init__()
        assert bits > 1, "RoundToBits is only used with bitwidth larger than 1."
        self.bits = bits
        self.epsilon = 1e-7

    def forward(self, input):
        """ extract the sign of each element """
        sign = torch.sign(input).detach()
        """ get the mantessa bits """
        input = torch.abs(input)
        scaling = torch.max(input).detach() + self.epsilon
        input = Clamp.apply( input/scaling ,0.0, 1.0 )
        """ round the mantessa bits to the required precision """
        input = Round.apply(input * (2.0**self.bits-1.0)) / (2.0**self.bits-1.0)
        return input * scaling * sign

In [64]:
class Round(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        # ctx.save_for_backward(input)
        return torch.round(input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        The backward behavior of the round function is defined as the identity function.
        """
        # input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        return grad_input

class Clamp(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input, min, max):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        # ctx.save_for_backward(input)
        return torch.clamp(input, min, max)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        The backward behavior of the clamp function is defined as the identity function.
        """
        # input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        return grad_input, None, None

In [65]:
class GaussianNoise(nn.Module):
    """Gaussian noise regularizer.
    Args:
        sigma (float, optional): relative standard deviation used to generate the
            noise. Relative means that it will be multiplied by the magnitude of
            the value your are adding the noise to. This means that sigma can be
            the same regardless of the scale of the vector.
        is_relative_detach (bool, optional): whether to detach the variable before
            computing the scale of the noise. If `False` then the scale of the noise
            won't be seen as a constant but something to optimize: this will bias the
            network to generate vectors with smaller values.
    """

    def __init__(self, sigma=0.1, is_relative_detach=False, inference=False, white=False):
        super().__init__()
        self.sigma = sigma
        self.is_relative_detach = is_relative_detach
        self.noise = torch.tensor(0).to("cuda")
        self.inference = inference
        self.white = white

    def forward(self, x):
        if (self.training and self.sigma > 0) or self.inference:
            scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
            if not self.white:
                sampled_noise = self.noise.repeat(*x.size()).float().normal_() * self.sigma
            else:
                sampled_noise = self.noise.repeat(*x.size()).float().random_(-1000000,1000000) * self.sigma / 1000000
            x = x + sampled_noise
        return x 

In [66]:
from torch.autograd import Function

class ActFn(Function):
	@staticmethod
	def forward(ctx, x, alpha, k):
		ctx.save_for_backward(x, alpha)
		# y_1 = 0.5 * ( torch.abs(x).detach() - torch.abs(x - alpha).detach() + alpha.item() )
		y = torch.clamp(x, min = 0, max = alpha.item())
		scale = (2**k - 1) / alpha
		y_q = torch.round( y * scale) / scale
		return y_q

	@staticmethod
	def backward(ctx, dLdy_q):
		# Backward function, I borrowed code from
		# https://github.com/obilaniu/GradOverride/blob/master/functional.py
		# We get dL / dy_q as a gradient
		x, alpha, = ctx.saved_tensors
		# Weight gradient is only valid when [0, alpha]
		# Actual gradient for alpha,
		# By applying Chain Rule, we get dL / dy_q * dy_q / dy * dy / dalpha
		# dL / dy_q = argument,  dy_q / dy * dy / dalpha = 0, 1 with x value range 
		lower_bound      = x < 0
		upper_bound      = x > alpha
		# x_range       = 1.0-lower_bound-upper_bound
		x_range = ~(lower_bound|upper_bound)
		grad_alpha = torch.sum(dLdy_q * torch.ge(x, alpha).float()).view(-1)
		return dLdy_q * x_range.float(), grad_alpha, None

In [67]:
class QuantizedLinear(nn.Linear):
    """ 
    A fully connected layer with its weight tensor and input tensor quantized. 
    """
    def __init__(self, in_features, out_features, bias=True, wbits=0, abits=0, pact=False, noise=0, half=False, white=False):
        super(QuantizedLinear, self).__init__(in_features, out_features, bias)
        self.quantize_w = TorchQuantize(wbits)
        self.quantize_a = TorchQuantize(abits)
        self.weight_rescale = np.sqrt(1.0/in_features) if (wbits == 1) else 1.0
        self.quantize_o = TorchQuantize(abits)
        # PACT
        self.quant = ActFn.apply
        self.k = abits
        if pact:
            self.alpha = nn.Parameter(torch.tensor(1.)) # trainable clipping factor for PACT
        self.pact = pact
        if half:
            self.noise = GaussianNoise(sigma=noise/2, inference=True, white=white)
        else:
            self.noise = GaussianNoise(sigma=noise, inference=True, white=white)

    def forward(self, input):
        """ 
        1. Quantize the input tensor
        2. Quantize the weight tensor
        3. Rescale via McDonnell 2018 (https://arxiv.org/abs/1802.08530)
        4. perform matrix multiplication 
        """
        if not self.pact:
              return F.linear(self.noise(self.quantize_a(input)), 
                        self.quantize_w(self.weight) * self.weight_rescale, 
                        self.bias)
        else:
            return F.linear(self.noise(self.quant(input, self.alpha, self.k)), 
                        self.quantize_w(self.weight) * self.weight_rescale, 
                        self.bias)

In [68]:
def replace_layer(module, name, noise=0.01, k=8):
    '''
    Replace linear layer to quantized layer
    '''
    # go through all attributes of module nn.module (e.g. network or layer) and put batch norms if present
    for attr_str in dir(module):
        target_attr = getattr(module, attr_str)
        if type(target_attr) == torch.nn.Linear:
            new = QuantizedLinear(target_attr.in_features, target_attr.out_features, True, 
                                     wbits=k, abits=k, noise=noise)
            setattr(module, attr_str, new)

    # iterate through immediate child modules. Note, the recursion is done by our code no need to use named_modules()
    for name, immediate_child_module in module.named_children():
        replace_layer(immediate_child_module, name, noise, k)

In [69]:
replace_layer(model, 'model', noise=0.01, k=1)

In [70]:
model
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): QuantizedLinear(
                in_features=768, out_features=768, bias=True
                (quantize_w): TorchQuantize(
                  (quantize): TorchBinarize()
                )
                (quantize_a): TorchQuantize(
                  (quantize): TorchBinarize()
                )
                (quantize_o): TorchQuantize(
                  (quantize): TorchBinarize()
                )
                (noise): GaussianNoise()
  

In [71]:
from datasets import load_dataset

dataset = load_dataset("klue", "ynat")

dataset

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 9107
    })
})

In [72]:
# load model&tokenizerx
tokenizer = BertTokenizerFast.from_pretrained(model_name, use_fast=True)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout"

In [73]:
def preprocess_function(examples):
    return tokenizer(examples['title'], max_length=max_length, truncation=True)

In [74]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset['train']

Dataset({
    features: ['guid', 'title', 'label', 'url', 'date', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 45678
})

In [75]:
len_dataset = len(encoded_dataset['train'])
len_validset = len(encoded_dataset['validation'])

print('#train =', len_dataset,
      '#valid =', len_validset
      )

total_batch_size = batch_size_per_device * torch.cuda.device_count()
n_total_iterations = int(len_dataset / total_batch_size * n_epochs)
n_warmup_steps = int(n_total_iterations * warmup_ratio)

print(
        '#total_iters =', n_total_iterations,
        '#warmup_iters =', n_warmup_steps,
      )

#train = 45678 #valid = 9107
#total_iters = 4282 #warmup_iters = 856


In [76]:
import os

output_dir = os.path.join("ynat")
logging_dir = os.path.join(output_dir, 'logs')
args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,

    # Model Save & Load
    save_strategy = "epoch", # 'steps'
    load_best_model_at_end=True,

    # Dataset
    num_train_epochs=n_epochs,
    per_device_train_batch_size= batch_size_per_device,
    per_device_eval_batch_size= batch_size_per_device,
    
    # Optimizer
    learning_rate=lr, # 5e-5
    weight_decay=0.01,  # 0
    warmup_steps=n_warmup_steps,

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,

    # Use mixed precision
    # mixed precision mode
    fp16=False,                                 
    #fp16_opt_level="02",  

    # Evaluation 
    metric_for_best_model='eval_f1',
    evaluation_strategy = "epoch",

    # Logging
    logging_dir=logging_dir,

    # Randomness
    seed=42,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [77]:
# datasets 라이브러리에서 제공하는 Evaluation metric의 리스트를 확인합니다.
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
len(metrics_list)
print(', '.join(metric for metric in metrics_list))

accuracy, bertscore, bleu, bleurt, brier_score, cer, character, charcut_mt, chrf, code_eval, comet, competition_math, coval, cuad, exact_match, f1, frugalscore, glue, google_bleu, indic_glue, mae, mahalanobis, mape, mase, matthews_correlation, mauve, mean_iou, meteor, mse, nist_mt, pearsonr, perplexity, poseval, precision, recall, rl_reliability, roc_auc, rouge, sacrebleu, sari, seqeval, smape, spearmanr, squad, squad_v2, super_glue, ter, trec_eval, wer, wiki_split, xnli, xtreme_s, BucketHeadP65/confusion_matrix, BucketHeadP65/roc_curve, Drunper/metrica_tesi, Felipehonorato/my_metric, GMFTBY/dailydialog_evaluate, GMFTBY/dailydialogevaluate, JP-SystemsX/nDCG, Josh98/nl2bash_m, KevinSpaghetti/accuracyk, NCSOFT/harim_plus, NikitaMartynov/spell-check-metric, NimaBoscarino/weat, Ochiroo/rouge_mn, Vertaix/vendiscore, Viona/infolm, Vlasta/pr_auc, abdusah/aradiawer, abidlabs/mean_iou, abidlabs/mean_iou2, angelina-wang/directional_bias_amplification, anz2/iliauniiccocrevaluation, bstrai/classif

In [78]:
# YNAT의 metric은 F1 score를 사용합니다.
metric_macrof1 = load_metric('f1')

def compute_metrics(eval_pred):
    predictions = eval_pred.predictions.argmax(-1)
    labels = eval_pred.label_ids
    return metric_macrof1.compute(predictions=predictions,
                                  references=labels, average='macro')

In [79]:
from transformers import default_data_collator
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [80]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: url, guid, title, date. If url, guid, title, date are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 45678
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4284
  Number of trainable parameters = 177858823
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,1.156800,1.373251,0.580557
2,0.796600,1.220386,0.634138
3,0.676700,1.129846,0.649780


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: url, guid, title, date. If url, guid, title, date are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9107
  Batch size = 32
Saving model checkpoint to ynat/checkpoint-1428
Configuration saved in ynat/checkpoint-1428/config.json
Model weights saved in ynat/checkpoint-1428/pytorch_model.bin
tokenizer config file saved in ynat/checkpoint-1428/tokenizer_config.json
Special tokens file saved in ynat/checkpoint-1428/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: url, guid, title, date. If url, guid, title, date are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation **

TrainOutput(global_step=4284, training_loss=0.9548048630259396, metrics={'train_runtime': 1357.8891, 'train_samples_per_second': 100.917, 'train_steps_per_second': 3.155, 'total_flos': 2010627694116300.0, 'train_loss': 0.9548048630259396, 'epoch': 3.0})

In [29]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: url, guid, title, date. If url, guid, title, date are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9107
  Batch size = 32


{'eval_loss': 0.7977203130722046,
 'eval_f1': 0.7183172904607441,
 'eval_runtime': 34.3171,
 'eval_samples_per_second': 265.378,
 'eval_steps_per_second': 8.305,
 'epoch': 3.0}

In [81]:
trainer.save_model('./checkpoint')

Saving model checkpoint to ./checkpoint
Configuration saved in ./checkpoint/config.json
Model weights saved in ./checkpoint/pytorch_model.bin
tokenizer config file saved in ./checkpoint/tokenizer_config.json
Special tokens file saved in ./checkpoint/special_tokens_map.json


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def predict(model, data_loader):
    print('start predict')

    model.eval()

    total_preds = []
    total_labels = []

    for step, batch in tqdm(enumerate(data_loader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)
        logit = outputs[0]

        logit = logit.detach().cpu().numpy()
        label = b_labels.cpu().numpy()
    
        total_preds += np.argmax(logit, axis=1).tolist()
        total_labels += label.tolist()

    f1 = f1_score(total_labels, total_preds, average='macro')
    avg = accuracy_score(total_labels, total_preds) * 100
    return f1, avg

In [ ]:
test = dataset['validation']
test_labels = torch.tensor(test['label'])

In [ ]:
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (KB):', size/1e3)
    os.remove('temp.p')
    return size

# compare the sizes
f=print_size_of_model(model,"yes")
print(sum(p.numel() for p in model.parameters()))

model:  yes  	 Size (KB): 711547.273
177858823


In [ ]:
def get_input_ids(data):
  document_bert = ["[CLS] " + str(s) + " [SEP]" for s in data]
  tokenized_texts = [tokenizer.tokenize(s) for s in tqdm(document_bert, "Tokenizing")]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(tokenized_texts, "Converting tokens to ids")]
  print("Padding sequences...")
  input_ids = pad_sequences(input_ids, maxlen=max_length, dtype='long', truncating='post', padding='post')
  return input_ids

def get_attention_masks(input_ids):
  attention_masks = []
  for seq in tqdm(input_ids, "Generating attention masks"):
      seq_mask = [float(i > 0) for i in seq]
      attention_masks.append(seq_mask)
  return attention_masks

In [ ]:
test_ids = torch.tensor(get_input_ids(test['title']))
test_masks = torch.tensor(get_attention_masks(test_ids))
test_labels = torch.tensor(test['label'])

In [ ]:
test_data = TensorDataset(test_ids, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size_per_device)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def predict(model, data_loader):
    print('start predict')

    model.eval()

    total_preds = []
    total_labels = []

    for step, batch in tqdm(enumerate(data_loader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)
        logit = outputs[0]

        logit = logit.detach().cpu().numpy()
        label = b_labels.cpu().numpy()
    
        total_preds += np.argmax(logit, axis=1).tolist()
        total_labels += label.tolist()

    f1 = f1_score(total_labels, total_preds, average='macro')
    avg = accuracy_score(total_labels, total_preds) * 100
    return f1, avg

In [ ]:
import time

def time_model_evaluation(model, tokenizer):
    eval_start_time = time.time()
    f1, avg_test_accuracy = predict(model, test_dataloader)
    eval_end_time = time.time()
    eval_duration_time = eval_end_time - eval_start_time
    print("Accuracy : {0:.4f}".format(avg_test_accuracy))
    print("f1 score : {0:.4f}".format(f1))
    print("Evaluate total time (seconds) : {0:.1f}".format(eval_duration_time))

# Evaluate the original FP32 BERT model
time_model_evaluation(model, tokenizer)

# Evaluate the INT8 BERT model after the dynamic quantization
time_model_evaluation(model_dynamic_quantized, tokenizer)